In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
#from utils import get_openai_api_key
#openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'wizardlm2:7b'
#os.environ["OPENAI_API_BASE"]='http://localhost:11434/v1'
#os.environ["OPENAI_API_KEY"]='NA'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
os.environ["LOCAL_EMBEDDINGS_MODEL"]="nomic-embed-text:v1.5"

### Agent - Busca no Contexto

In [4]:
from search.context import utils
from crewai_tools import tool

# Embeddings Model
embeddings_model = "nomic-embed-text:v1.5"
# Vectore Store path
path_books = "./vector-store/chroma/books"

@tool("Pesquisar em Livros")
def library_tool(text: str) -> str:
    """Retorna livros que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    _, db = utils.retriver_context(embeddings_model=embeddings_model, path_books=path_books, vector_store='chroma')
    documents = db.similarity_search(text)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source']
        page = row.metadata['page']
        result += f"**Trecho de {source.replace('./books/import/','').replace('.pdf','')}** - página {page}: \n"
        result += f"{page_content}\n\n"
    return result

In [5]:
library_agent = Agent(
    role="Especialista em Pesquisa em Biblioteca",
    goal="Encontre os trechos de livros mais relevantes relacionados ao texto fornecido.",
    backstory=(
        "Você atualmente é encarregado de usar a library_tool "
        "para encontrar trechos relevantes de livros relacionados "
        "para um texto fornecido. "
        "Seu objetivo é garantir que os trechos que você fornece "
        "são altamente relevantes e úteis para análises posteriores. "
    ),
    tools=[library_tool],
    allow_delegation=False,
    verbose=True
)



commentary_agent = Agent(
    role="Especialista em Comentário Literário",
    goal="Forneça comentários perspicazes sobre os trechos fornecidos pelo Especialista em Pesquisa da Biblioteca.",
    backstory=(
        "Você atualmente é encarregado de fornecer comentários perspicazes "
        "em trechos de livros encontrados pelo Especialista em Pesquisa da Biblioteca. "
        "Seu objetivo é garantir que seu comentário seja direto "
        "relacionado ao texto fornecido e ajuda a melhorar "
        "compreensão e apreciação do material."
    ),
    verbose=True,
)


### Task - Busca no contexto

In [6]:
library_search_task = Task(
    description=(
        "O cliente forneceu o seguinte texto para análise:\n"
        "{text}\n\n"
        "Sua tarefa é usar a library_tool para encontrar o mais relevante" 
        "trechos de livros relacionados a este texto. Certifique-se de que os trechos "
        "que você fornece são altamente relevantes e úteis para análises posteriores."
    ),
    expected_output=(
        "Trechos de livros diretamente relacionados ao texto fornecido. "
        "Cada trecho deve ser claramente citado com sua fonte, incluindo o título do livro "
        "autor e número da página. Os trechos devem ajudar a fornecer insights mais profundos "
        "ou contexto adicional ao texto fornecido."
    ),
    tools=[library_tool],
    agent=library_agent,
)

In [7]:
literary_commentary_task = Task(
    description=(
        "Sua tarefa é citar os trechos retornados pelo"
        "Especialista em Pesquisa da Biblioteca para a pesquisa {text}"
        "e fornecer comentários perspicazes sobre eles. "
        "Certifique-se de que seu comentário esteja diretamente relacionado ao texto buscado "
        "e ajude a melhorar a compreensão e apreciação do material. "
        "Responda em português."
    ),
    expected_output=(
        "Texto original fornecido pelo cliente, "
        "seguido pelos trechos de livros fornecidos pelo Especialista em "
        "Pesquisa da Biblioteca. Por fim, um comentário detalhado sobre os trechos fornecidos. "
        "Cada comentário deve explicar como o trecho se relaciona com o texto original "
        "e quais insights ou adicionais contexto que ele fornece. "
    ),
    agent=commentary_agent,
    context=[library_search_task],
)


In [8]:
crew = Crew(
  agents=[library_agent, commentary_agent],
  tasks=[library_search_task, literary_commentary_task],
  verbose=2,
  memory=False,
  #language="pt"
)

In [9]:
inputs = {
    "text": """O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas."""
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Especialista em Pesquisa em Biblioteca
 [INFO]: == Starting Task: O cliente forneceu o seguinte texto para análise:
O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas.

Sua tarefa é usar a library_tool para encontrar o mais relevantetrechos de livros relacionados a este texto. Certifique-se de que os trechos que você fornece são altamente relevantes e úteis para análises posteriores.


> Entering new CrewAgentExecutor chain...
Preciso pesquisar em livros que contenham conteúdo relacionado ao texto fornecido para encontrar trechos relevantes.

Action: Pesquisar em Livros
Action Input: {"text": "vivam em sua comunidade de maneira digna das boas-novas de Cristo"} 

**Trecho de Evangelho Maltrapilho - Brennan Manni

In [10]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_search_task))



**Tarefa:** 

O cliente forneceu o seguinte texto para análise:
O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas.

Sua tarefa é usar a library_tool para encontrar o mais relevantetrechos de livros relacionados a este texto. Certifique-se de que os trechos que você fornece são altamente relevantes e úteis para análises posteriores. 
 

**Saída:** 

**Trecho de Evangelho Maltrapilho - Brennan Manning** - página 77: 
comigo? 
Você alega saber o que compartilhamos qu ando Jesus retirou-se para um lugar 
deserto e passou a noite na encosta de uma colina comigo? Você sabe de onde veio a inspiração de lavar os pés dos doze discíp ulos? Você entende que, motivado apenas 
pelo amor, seu Deus tornou-se seu escravo no cenáculo? 
Você ressente a ordem divina dada a Abr aão, de oferecer seu único filho, Isaque, 
no monte Moriá? Você sente alívio quando o anjo intervém, a mão de Abraão é detida

**Trecho de Evangelho Maltrapilho - Brennan Manning** - página 99: 
ordem de Jesus: "Tende bom ânimo! Sou eu. Não temais!" (Mc 6:50) e às palavras do discípulo 
que Jesus amava: "No amor não existe medo; ante s, o perfeito amor lança fora o medo. Ora, o 
medo produz tormento; logo, aquele que teme não é aperfeiçoado no amor" (1Jo 4:18). Uma 
pequena maltrapilha chamada Amy Welborn dá uma voz à cambada dos esfarrapados: 
O amor pode lançar fora o medo, mas alguns  de nós devem tomar o desafio de João na

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill** - página 24: 
contra nós toda a oposição que ê capaz de arregimentar ”. 
— R. Sibbes.  
“Busquei entre eles um homem ”. 
— Ezequiel 22.30.  
“Elias era homem ”. 
— Tiago 5.17.

**Trecho de O Peregrino - John Bunyan** - página 92: 
34:20-24).
— Os fariseus hipócritas — disse Cristão — também praticavam essa mesma
religião. Seu pretexto eram as longas orações, mas sua meta era tomar as casas
das viúvas, e por isso Deus dedicou-lhes maior condenação em seu juízo (Lc
20:46-47).
— Judas, o diabo — acrescentou ele — também era dessa religião. Ele era
religioso só com vistas a possuir a bolsa e o que nela havia, mas se perdeu, foi
renegado e tornou-se o próprio filho da perdição.


In [11]:
Markdown(print(inputs['text']))
Markdown(result)

O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas.


**Trecho de Evangelho Maltrapilho - Brennan Manning** - página 77: 
comigo? 
Você alega saber o que compartilhamos qu ando Jesus retirou-se para um lugar 
deserto e passou a noite na encosta de uma colina comigo? Você sabe de onde veio a inspiração de lavar os pés dos doze discíp ulos? Você entende que, motivado apenas 
pelo amor, seu Deus tornou-se seu escravo no cenáculo? 
Você ressente a ordem divina dada a Abr aão, de oferecer seu único filho, Isaque, 
no monte Moriá? Você sente alívio quando o anjo intervém, a mão de Abraão é detida

**Trecho de Evangelho Maltrapilho - Brennan Manning** - página 99: 
ordem de Jesus: "Tende bom ânimo! Sou eu. Não temais!" (Mc 6:50) e às palavras do discípulo 
que Jesus amava: "No amor não existe medo; ante s, o perfeito amor lança fora o medo. Ora, o 
medo produz tormento; logo, aquele que teme não é aperfeiçoado no amor" (1Jo 4:18). Uma 
pequena maltrapilha chamada Amy Welborn dá uma voz à cambada dos esfarrapados: 
O amor pode lançar fora o medo, mas alguns  de nós devem tomar o desafio de João na

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill** - página 24: 
contra nós toda a oposição que ê capaz de arregimentar ”. 
— R. Sibbes.  
“Busquei entre eles um homem ”. 
— Ezequiel 22.30.  
“Elias era homem ”. 
— Tiago 5.17.

**Trecho de O Peregrino - John Bunyan** - página 92: 
34:20-24).
— Os fariseus hipócritas — disse Cristão — também praticavam essa mesma
religião. Seu pretexto eram as longas orações, mas sua meta era tomar as casas
das viúvas, e por isso Deus dedicou-lhes maior condenação em seu juízo (Lc
20:46-47).
— Judas, o diabo — acrescentou ele — também era dessa religião. Ele era
religioso só com vistas a possuir a bolsa e o que nela havia, mas se perdeu, foi
renegado e tornou-se o próprio filho da perdição.

Comentário: Cada trecho fornecido pelo Especialista em Pesquisa da Biblioteca traz à tona aspectos essenciais relacionados à fé, amor e coragem. Os relatos sobre a inspiração divina, a importância do amor para afastar o medo e a necessidade de buscar homens de fé são elementos fundamentais para viver de acordo com os princípios cristãos. Além disso, a referência aos fariseus hipócritas e a Judas, o diabo, destaca a importância da sinceridade e da verdadeira devoção na prática da fé. Esses trechos fornecem uma visão mais profunda e significativa sobre como a vivência cristã deve ser baseada em amor, coragem e autenticidade.

In [12]:
print(crew.usage_metrics)

{'total_tokens': 32623, 'prompt_tokens': 24002, 'completion_tokens': 8621, 'successful_requests': 22}
